In [1]:
from notes import show

# Uguaglianza ed ereditarietà

Questa breve nota discute i limiti imposti dall'ereditarietà al soddisfacimento del contratto imposto dal metodo [`equals`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/Object.html#equals(java.lang.Object)) della classe `Object`. 

La nota segue l'esposizione della *Sezione 7.9.3* del testo "Program Development in Java" (di Barbara H. Liskov *et al.*) e dell'*Item 10* del testo "Effective Java" (di Joshua Bloch).

## Simmetria 

L'implementazione *ovvia* non funziona: se un sottotipo aggiunge un "valore", non basta controllare con `instanceof` e verificare che siano uguali i valori in gioco…

Consideriamo il seguente tipo `T`

In [2]:
show('it/unimi/di/prog2/notes/uee/sym/T.java', fragment = 'class T', highlight = 'equals(Object')

e il suo sottotipo `S` 

In [3]:
show('it/unimi/di/prog2/notes/uee/sym/S.java', fragment = 'class S', highlight = 'equals(Object')

Entrambi i tipi sono dotati di una implementazione di `equals` (qui evidenziata in giallo) che, dopo aver verificato il tipo dell'argomento `o` procedono all'ovvio controllo dell'identità dei valori del/i campo/i corrispondente/i.

Tale implementazione viola la proprietà di **simmetria**, ossia $t \sim s$ non implica $s \sim t$, come dimostrato dal seguente metodo `main`

In [4]:
show('it/unimi/di/prog2/notes/uee/MainSym.java', fragment = 'main(String')

che produce l'output

In [5]:
! java -cp ../build/classes/java/main/ it.unimi.di.prog2.notes.uee.MainSym

true
false


Quel che accade è che `t instanceof S` è *falso*, ma `s instanceof T` è *vero*, per cui l'implementazione di `equals` di `S` non sa cosa fare con `t`, mentre quella di `T` funziona con `s` (ovviamente trascurando `b`, di cui ignora l'esistenza).

## Transitività

Non è facile aggiustare l'implementazione di `equals` di `S` gestendo accortamente il caso in cui `o` ha il tipo effettivo di `T` imitando il suo comportamento, ossia "scordandosi" di `b`, come (nella parte evidenziata) nel seguente codice

In [6]:
show('it/unimi/di/prog2/notes/uee/trans/S.java', fragment = 'equals(Object', highlight = [42, 43])

Così facendo infatti, la simmetria è rispettata, ma risulterà violata la proprietà **transitiva**, ovvero anche avendo $s \sim t$ e $t \sim u$ non è detto che $s \sim u$. Questo è dimostrato dalla seguente porzione di codice

In [7]:
show('it/unimi/di/prog2/notes/uee/MainTrans.java', fragment = 'main(String')

che produce l'output

In [8]:
! java -cp ../build/classes/java/main/ it.unimi.di.prog2.notes.uee.MainTrans

true
true
false


## Il principio di sostituzione

Si potrebbe pensare che il problema sia `instanceof` che è antisimmetrico, per cui una soluzione che viene talvolta suggerita è di sostituire nell'implementazione di `equals` di `T` la condizione `o instanceof T` con la condizione `getClass() == o.getClass()` che è simmetrica (e risulta vera se e solo se gli oggetti hanno identico tipo concreto).

In [9]:
show('it/unimi/di/prog2/notes/uee/lsp/T.java', fragment = 'equals(Object')

Si può applicare la stessa scelta anche a `S`, sebbene questo sia utile solo qualora tale classe venisse estesa.

In [10]:
show('it/unimi/di/prog2/notes/uee/lsp/S.java', fragment = 'equals(Object')

In questo modo la simmetria e la transitività sono rispettate perché `equals` funziona solo per oggetti del medesimo tipo, avendo valore `false` per oggetti di tipo diverso (anche se sottotipi l'uno dell'altro).

Questo però viola il **principio di sostituzione**! 

Per cogliere meglio questo aspetto consideriamo `R`, una ulteriore estensione di `T` che non aggiunga alcun "valore", ma solo un "comportamento" e che, pertanto, non ridefinisca neppure `equals`.

In [11]:
show('it/unimi/di/prog2/notes/uee/lsp/R.java', fragment = 'class R')

Supponiamo ora di sviluppare una classe di utilità con un metodo `isSmall` che restituisca `true` per tutti gli oggetti per cui `a` è `1`, oppure `2` e supponiamo (sebbene questo sia estremamente innaturale e inefficiente) di implementare tale metodo verificando l'appartenenza a un insieme `SMALLS` che contiene gli unici due oggetti che soddisfano tale requisito.

In [12]:
show('it/unimi/di/prog2/notes/uee/lsp/Client.java', fragment = 'class Client')

La seguente porzione di codice mostra che il confronto tra oggetti di tipo diverso (anche se uno è sottotipo dell'altro) restituisce `false` (il che risolve i problemi di simmetria e transitività); mostra però anche che sostituendo il sottotipo `R` a `T` (vedi il codice evidenziato) il comportamento cambia!

In [13]:
show('it/unimi/di/prog2/notes/uee/MainLSP.java',  fragment = 'main(String', highlight = [41, 42, 43])

L'output infatti è

In [14]:
! java -cp ../build/classes/java/main/ it.unimi.di.prog2.notes.uee.MainLSP

false
false
true
false


Questo accade perché l'implementazione di `Set` si basa sul metodo `equals` per decidere se un oggetto appartiene all'insieme, ma gli elementi di `SMALLS` (che sono di tipo `T`) non risultano mai uguali a oggetti di tipo `R`!